In [ ]:
!pip install liwc
!pip install nltk
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install -U imbalanced-learn
!pip install pyspellchecker
!pip install svgutils
!pip install -U kaleido
!pip install -U plotly psutil requests orca
!pip install pscript

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import gensim
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from textblob import TextBlob
from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from IPython.display import display
from nltk.probability import FreqDist
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import os

# **Extraer datos de los usarios**

In [ ]:
def cargar_documentos(ruta):
    documentos = []

    for archivo in os.listdir(ruta):
        if archivo.endswith(".tsv"):
            # Leer el archivo CSV usando pandas
            df = pd.read_csv(os.path.join(ruta, archivo), sep='\t')

            # Verificar si la columna "TWEET_TEXT" existe en el DataFrame
            if 'TWEET_TEXT' in df.columns:
                # Obtener solo la columna "tweet"
                tweets = df['TWEET_TEXT'].tolist()

                # Agregar los tweets a la lista de documentos
                documentos.extend(tweets)
    return documentos

def cargar_documentos_csv(ruta):
    documentos = []

    for archivo in os.listdir(ruta):
        if archivo.endswith(".csv"):
            # Leer el archivo CSV usando pandas
            df = pd.read_csv(os.path.join(ruta, archivo), delimiter=',')

            # Verificar si la columna "tweet" existe en el DataFrame
            if 'tweet' in df.columns:
                # Obtener solo la columna "tweet"
                tweets = df['tweet'].tolist()

                # Agregar los tweets a la lista de documentos
                documentos.extend(tweets)

    return documentos

In [ ]:
# Ruta de los usarios dignosticados con Ansiedad, Depresión y los usarios de Control

ruta_ans = '/content/drive/MyDrive/Experimentación Python Tesis /Experimento por agrupación de usuarios/Usuarios con ansiedad/'
ruta_dep = '/content/drive/MyDrive/Experimentación Python Tesis /Experimento por agrupación de usuarios/Usuarios con depresión/'
ruta_cont = '/content/drive/MyDrive/Experimentación Python Tesis /Experimento por agrupación de usuarios/Usuario de control/'
#Ansiedad
Ans = cargar_documentos_csv(ruta_ans)
# Crear un DataFrame usando pandas
data_ansiedad = pd.DataFrame({'TWEET_TEXT': Ans })
#Depresióm
Dep = cargar_documentos(ruta_dep)
# Crear un DataFrame usando pandas
data_depresion = pd.DataFrame({'TWEET_TEXT': Dep})
#Control
Cont = cargar_documentos(ruta_cont)
# Crear un DataFrame usando pandas
data_control = pd.DataFrame({'TWEET_TEXT': Cont})

***DATOS CONTROL***

In [ ]:
data_control.info()
data_control.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239127 entries, 0 to 239126
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   TWEET_TEXT  239127 non-null  object
dtypes: object(1)
memory usage: 1.8+ MB


,TWEET_TEXT
0,Ottavis tiene un reality. Esto está sucediendo...
1,RT @bbimbi: Lula no irá a Curitiba a entregars...
2,RT @suzyqiu: 📦🔥 #CajaDeHerramientas capítulo 7...
3,"Ayer terminé de ver La Casa de Papel, y no le ..."
4,RT @latidosdelrock: Iluminame en esta eterna n...
5,"Cuando uno tiene sed, pero el agua no está cer..."
6,RT @rechimuzziok: Presidente ReySol Handel se ...
7,@Lara_Imhof Bravo amiga 👏💜
8,RT @FrasesSilvio: “Hay locuras que hicieron el...
9,Me siento como en un programa de Capusotto. ht...


***DATOS DEPRESIÓN***

In [ ]:
data_depresion.info()
data_depresion.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189665 entries, 0 to 189664
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   TWEET_TEXT  189665 non-null  object
dtypes: object(1)
memory usage: 1.4+ MB


,TWEET_TEXT
0,Empiezo a trabajar en una semana y no se de do...
1,Pronto lo sabré.
2,Me quiero morir si no estás donde estoy.
3,Que asco de vida.
4,Unos cuantos y a la cama.
5,Ojala todas las mierdas que me meto acaben ráp...
6,RT @YourFaveMuse: Nada peor que una mujer cria...
7,Ninguna actividad te entretiene.
8,"La ansiedad se ha apoderado de mi vida, y grac..."
9,"RT @beiirr97_: hoy tengo un bajón, que no me a..."


***DATOS ANSIEDAD***

In [ ]:
data_ansiedad.info()
data_ansiedad.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64906 entries, 0 to 64905
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   TWEET_TEXT  64906 non-null  object
dtypes: object(1)
memory usage: 507.2+ KB


,TWEET_TEXT
0,@USUARIO JAJAJAJAJA esa es una historia mal co...
1,@USUARIO al menos no atrasé a mi causa con su ...
2,que sería sino
3,me gusta la calle cuando hay gente
4,"odio que me manden audios de más de 2min, teng..."


**PREPARACIÓN DE DATOS Y LIMPIEZA**

In [ ]:
# Obtener las palabras vacías en español

nltk.download('stopwords')
stopwords_en = set(stopwords.words('spanish')) #las palabras vacías en español utilizando la biblioteca NLTK (palabras que no aportan mucho significado al texto)

# **Limpiar**

In [ ]:
import re
import string


def removeHTML(text):
    if isinstance(text, str):  # Verificar si es de tipo string
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', text)
        return cleantext
    else:
        return text  # Devolver el valor original si no es de tipo string

def clean(text):
    text = text.lower()  # Convertir el texto a minúsculas
    # Varios pasos de limpieza usando sub() para sustituir patrones encontrados por cadenas vacías o espacios, según corresponda
    text = re.sub('\[.*?\]', '', text)  # Eliminar texto entre corchetes
    text = re.sub("\\W"," ",text)  # Sustituir caracteres no alfanuméricos por espacios
    text = re.sub('https?://\S+|www\.\S+', '', text)  # Eliminar URLs
    text = re.sub('<.*?>+', '', text)  # Eliminar etiquetas HTML, redundante debido a la función removeHTML
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Eliminar puntuación
    text = re.sub('\n', '', text)  # Eliminar saltos de línea
    text = re.sub('\w*\d\w*', '', text)  # Eliminar palabras que contienen números
    text = re.sub(r'#\w+', '', text)  # Eliminar hashtags
    text = re.sub(r'@\w+', '', text)  # Eliminar menciones
    text = re.sub(r'\b[A-Z][a-z]+\b', '', text)  # Eliminar palabras que empiezan con mayúsculas (posiblemente nombres propios)
    text = re.sub(r'[^\w\s]+', '', text)  # Eliminar caracteres especiales
    text = re.sub(r'\b\w{1,2}\b|\d+', '', text)  # Eliminar palabras de 1 o 2 letras y números
    # Proceso de eliminación de emojis y caracteres especiales listados manualmente
    emojis = re.findall(r'[^\w\s,]', text)
    cleaned_emojis = [emoji for emoji in emojis if emoji not in [...]]  # Lista de emojis y caracteres a mantener o eliminar
    for emoji in cleaned_emojis:
        text = text.replace(emoji, '')  # Eliminar emojis y caracteres seleccionados
    return text  # Devolver el texto limpio



# Limpieza de los datos

In [ ]:
# Control
data_control['TWEET_TEXT'] = data_control.TWEET_TEXT.apply(lambda x : removeHTML(x))
data_control['TWEET_TEXT'] = data_control.TWEET_TEXT.apply(lambda x : clean(x))
#Depresión
data_depresion['TWEET_TEXT'] = data_depresion.TWEET_TEXT.apply(lambda x : removeHTML(x))
data_depresion['TWEET_TEXT'] = data_depresion.TWEET_TEXT.apply(lambda x : clean(x))
#Ansiedad
data_ansiedad['TWEET_TEXT'] = data_ansiedad.TWEET_TEXT.apply(lambda x : removeHTML(x))
data_ansiedad['TWEET_TEXT'] = data_ansiedad.TWEET_TEXT.apply(lambda x : clean(x))

#Stop Words

In [ ]:
def Stopword(text):
  token_text = nltk.word_tokenize(text)  # Tokenizar el texto en palabras individuales
  remover_palab = [word for word in token_text if word not in stopwords_en]  # Filtrar las palabras, eliminando las stopwords
  join_text = ' '.join(remover_palab)  # Unir las palabras filtradas en una cadena de texto
  return join_text  # Devolver el texto procesado


In [ ]:
# Control
data_control['TWEET_TEXT'] = data_control.TWEET_TEXT.apply(lambda x : Stopword(x))

#Depresión
data_depresion['TWEET_TEXT'] = data_depresion.TWEET_TEXT.apply(lambda x : Stopword(x))

#Ansiedad
data_ansiedad['TWEET_TEXT'] = data_ansiedad.TWEET_TEXT.apply(lambda x : Stopword(x))


# **LIWC: Spanish_LIWC2007_Dictionary**

In [ ]:
import liwc
liwcPath= r'/content/drive/MyDrive/Experimentación Python Tesis /Diccionarios español/Spanish_LIWC2007_Dictionary.dic'
# Cargar el diccionario y obtener la función de análisis y nombres de categorías
parse, category_names = liwc.load_token_parser(liwcPath)

# Mostrar las categorías disponibles en el diccionario
print(f'Categorias del diccionario: {category_names}')

Categorias del diccionario: ['Funct', 'TotPron', 'PronPer', 'Yo', 'Nosotro', 'TuUtd', 'ElElla', 'Ellos', 'PronImp', 'Articulo', 'Verbos', 'VerbAux', 'Pasado', 'Present', 'Futuro', 'Adverb', 'Prepos', 'Conjunc', 'Negacio', 'Cuantif', 'Numeros', 'Maldec', 'verbYO', 'verbTU', 'verbNOS', 'verbosEL', 'verbELLOS', 'Subjuntiv', 'VosUtds', 'formal', 'informal', 'verbVos', 'Social', 'Familia', 'Amigos', 'Humanos', 'Afect', 'EmoPos', 'EmoNeg', 'Ansiedad', 'Enfado', 'Triste', 'MecCog', 'Insight', 'Causa', 'Discrep', 'Tentat', 'Certeza', 'Inhib', 'Incl', 'Excl', 'Percept', 'Ver', 'Oir', 'Sentir', 'Biolog', 'Cuerpo', 'Salud', 'Sexual', 'Ingerir', 'Relativ', 'Movim', 'Espacio', 'Tiempo', 'Trabajo', 'Logro', 'Placer', 'Hogar', 'Dinero', 'Relig', 'Muerte', 'Asentir', 'NoFluen', 'Relleno']


# **Usar LIWC para extraer las caracteristicas del texto**

**Datos de control**

In [ ]:
corpus_control = []  # Inicializa una lista vacía para almacenar los tweets transformados en categorías LIWC
words_control = []  # Inicializa una lista vacía para almacenar listas de categorías LIWC por tweet

# Itera sobre cada tweet en el DataFrame data_control
for i in range(0, len(data_control)):
    # Limpia el texto del tweet, manteniendo solo caracteres alfanuméricos y reemplazando otros con espacios
    review = re.sub('[^a-zA-Z0-9]', ' ', data_control['TWEET_TEXT'][i])
    review = review.lower()  # Convierte el texto a minúsculas para estandarización
    review = review.split()  # Divide el texto en una lista de palabras individuales

    # Utiliza la función parse para obtener las categorías LIWC correspondientes a cada palabra
    review = [category for token in review for category in parse(token)]
    statements = ' '.join(review)  # Une las categorías LIWC en una cadena de texto, separadas por espacios

    corpus_control.append(statements)  # Añade la cadena de texto de categorías LIWC al corpus_control
    words_control.append(review)  # Añade la lista de categorías LIWC a words_control



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Crear una instancia de CountVectorizer
count_vect = CountVectorizer()

# Aplicar CountVectorizer al corpus
count_train = count_vect.fit_transform(corpus_control)

# Imprimir el tamaño del vocabulario construido por CountVectorizer
print("El tamaño del vocabulario es: ", len(count_vect.vocabulary_))

X = count_train  # Matriz de conteo

# Obtener índices de filas no nulas en X
non_null_indices = np.any(X.toarray(), axis=1)

# Filtrar X y y para eliminar filas nulas
X = X[non_null_indices]

features = count_vect.get_feature_names_out()

# Crear un DataFrame con la matriz de conteo
df_count_C = pd.DataFrame(X.toarray(), columns=features)
df_count_C

El tamaño del vocabulario es:  73


,adverb,afect,amigos,ansiedad,articulo,asentir,biolog,causa,certeza,conjunc,...,verbaux,verbellos,verbnos,verbos,verbosel,verbtu,verbvos,verbyo,vosutds,yo
0,1,1,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
1,0,2,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219706,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219707,0,0,1,0,0,0,0,0,1,0,...,0,0,0,2,2,0,0,0,0,0
219708,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219709,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import plotly.express as px
import pandas as pd

df_C = df_count_C.sum().reset_index()

fig = px.bar(df_C, x='index', y=0,
             title='Frecuencia de categorias de LIWC - usarios de control',
             labels={'index': 'Palabra', 0: 'Frecuencia'},
             width=1600, height=700)

# Ajustes visuales
fig.update_traces(marker_color='skyblue', marker_line_color='black', marker_line_width=1, opacity=0.7)
fig.update_layout(xaxis_tickangle=-90, xaxis=dict(type='category'))

# Ajustes de diseño
fig.update_layout(
    margin=dict(l=50, r=50, t=80, b=50),  # Márgenes para mejorar la visualización
    showlegend=False  # No mostrar la leyenda
)

# Mostrar el gráfico
fig.show()


In [ ]:
# Las categorias más frecuentes en los tweets de los usarios de control
# Sumar las frecuencias de cada categoría
sums = df_count_C.sum()

# Crear un DataFrame con las sumas ordenadas de forma descendente
df_top_C = pd.DataFrame({'Categoría': sums.index, 'Frecuencia': sums.values}).sort_values(by='Frecuencia', ascending=False)
df_top_C.head(10)

,Categoría,Frecuencia
33,meccog,205231
66,verbos,192928
48,relativ,183757
22,funct,157426
45,present,152701
1,afect,151118
54,social,136545
17,emopos,98710
57,tiempo,85462
42,percept,71543


**Datos de depresión**

In [ ]:
corpus_depresion = []  # Inicializa una lista vacía para almacenar los tweets transformados en categorías LIWC
words_depresión = []  # Inicializa una lista vacía para almacenar listas de categorías LIWC por tweet

# Itera sobre cada tweet en el DataFrame data_depresion
for i in range(0, len(data_depresion)):
    # Limpia el texto del tweet, manteniendo solo caracteres alfanuméricos y reemplazando otros con espacios
    review = re.sub('[^a-zA-Z0-9]', ' ', data_depresion['TWEET_TEXT'][i])
    review = review.lower()  # Convierte el texto a minúsculas para estandarización
    review = review.split()  # Divide el texto en una lista de palabras individuales

    # Utiliza la función parse para obtener las categorías LIWC correspondientes a cada palabra
    review = [category for token in review for category in parse(token)]
    statements = ' '.join(review)  # Une las categorías LIWC en una cadena de texto, separadas por espacios

    corpus_depresion.append(statements)  # Añade la cadena de texto de categorías LIWC al corpus_depresion
    words_depresión.append(review)  # Añade la lista de categorías LIWC a words_depresión



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Crear una instancia de CountVectorizer
count_vect = CountVectorizer()

# Aplicar CountVectorizer al corpus
count_train = count_vect.fit_transform(corpus_depresion)

# Imprimir el tamaño del vocabulario construido por CountVectorizer
print("El tamaño del vocabulario es: ", len(count_vect.vocabulary_))

X = count_train  # Matriz de conteo

# Obtener índices de filas no nulas en X
non_null_indices = np.any(X.toarray(), axis=1)

# Filtrar X y y para eliminar filas nulas
X = X[non_null_indices]

features = count_vect.get_feature_names_out()

# Crear un DataFrame con la matriz de conteo
df_count_D = pd.DataFrame(X.toarray(), columns=features)
df_count_D

El tamaño del vocabulario es:  73


,adverb,afect,amigos,ansiedad,articulo,asentir,biolog,causa,certeza,conjunc,...,verbaux,verbellos,verbnos,verbos,verbosel,verbtu,verbvos,verbyo,vosutds,yo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167695,0,3,0,0,0,0,0,1,0,0,...,0,0,0,2,1,0,0,1,0,0
167696,0,0,1,0,0,0,0,0,0,0,...,0,1,0,2,0,0,0,0,0,0
167697,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,0,0
167698,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import plotly.express as px
import pandas as pd

df_D = df_count_D.sum().reset_index()

fig = px.bar(df_D, x='index', y=0,
             title='Frecuencia de categorias de LIWC - usarios de Depresión',
             labels={'index': 'Palabra', 0: 'Frecuencia'},
             width=1600, height=700)

# Ajustes visuales
fig.update_traces(marker_color='skyblue', marker_line_color='black', marker_line_width=1, opacity=0.7)
fig.update_layout(xaxis_tickangle=-90, xaxis=dict(type='category'))

# Ajustes de diseño
fig.update_layout(
    margin=dict(l=50, r=50, t=80, b=50),  # Márgenes para mejorar la visualización
    showlegend=False  # No mostrar la leyenda
)

# Mostrar el gráfico
fig.show()


In [ ]:
# Las categorias más frecuentes en los tweets de los usarios de depresión
# Sumar las frecuencias de cada categoría
sums = df_count_D.sum()

# Crear un DataFrame con las sumas ordenadas de forma descendente
df_top_D = pd.DataFrame({'Categoría': sums.index, 'Frecuencia': sums.values}).sort_values(by='Frecuencia', ascending=False)
df_top_D.head(10)

,Categoría,Frecuencia
33,meccog,233888
66,verbos,187818
22,funct,157915
45,present,153291
1,afect,150095
48,relativ,140118
54,social,112082
16,emoneg,87089
6,biolog,72314
30,insight,69311


**ANSIEDAD**

In [ ]:
corpus_ansiedad = []  # Inicializa una lista vacía para almacenar los tweets transformados en categorías LIWC.
words_ansiedad = []  # Inicializa una lista vacía para almacenar listas de categorías LIWC por tweet.

# Itera sobre cada tweet en el DataFrame data_ansiedad.
for i in range(0, len(data_ansiedad)):
    # Limpia el texto del tweet, eliminando todo excepto caracteres alfanuméricos y reemplazándolos con espacios.
    review = re.sub('[^a-zA-Z0-9]', ' ', data_ansiedad['TWEET_TEXT'][i])
    review = review.lower()  # Convierte el texto a minúsculas para una mayor consistencia.
    review = review.split()  # Divide el texto en palabras individuales.

    # Convierte cada palabra en las categorías LIWC correspondientes, si están disponibles.
    review = [category for token in review for category in parse(token)]
    statements = ' '.join(review)  # Une las categorías LIWC en un solo string, separadas por espacios.

    corpus_ansiedad.append(statements)  # Añade la cadena de texto de categorías LIWC al corpus_ansiedad.
    words_ansiedad.append(review)  # Añade la lista de categorías LIWC obtenidas a words_ansiedad.



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Crear una instancia de CountVectorizer
count_vect = CountVectorizer()

# Aplicar CountVectorizer al corpus
count_train = count_vect.fit_transform(corpus_ansiedad)

# Imprimir el tamaño del vocabulario construido por CountVectorizer
print("El tamaño del vocabulario es: ", len(count_vect.vocabulary_))

X = count_train  # Matriz de conteo

# Obtener índices de filas no nulas en X
non_null_indices = np.any(X.toarray(), axis=1)

# Filtrar X y y para eliminar filas nulas
X = X[non_null_indices]

features = count_vect.get_feature_names_out()

# Crear un DataFrame con la matriz de conteo
df_count_A = pd.DataFrame(X.toarray(), columns=features)
df_count_A

El tamaño del vocabulario es:  73


,adverb,afect,amigos,ansiedad,articulo,asentir,biolog,causa,certeza,conjunc,...,verbaux,verbellos,verbnos,verbos,verbosel,verbtu,verbvos,verbyo,vosutds,yo
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,3,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58737,1,1,0,0,0,1,0,1,0,0,...,0,0,1,8,0,0,0,0,0,0
58738,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58739,0,0,0,0,0,0,1,2,1,0,...,1,0,0,3,1,0,0,0,0,0
58740,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
import plotly.express as px
import pandas as pd

df_A = df_count_A.sum().reset_index()

fig = px.bar(df_A, x='index', y=0,
             title='Frecuencia de categorias de LIWC - usarios de Ansiedad',
             labels={'index': 'Palabra', 0: 'Frecuencia'},
             width=1600, height=700)

# Ajustes visuales
fig.update_traces(marker_color='skyblue', marker_line_color='black', marker_line_width=1, opacity=0.7)
fig.update_layout(xaxis_tickangle=-90, xaxis=dict(type='category'))

# Ajustes de diseño
fig.update_layout(
    margin=dict(l=50, r=50, t=80, b=50),  # Márgenes para mejorar la visualización
    showlegend=False  # No mostrar la leyenda
)

# Mostrar el gráfico
fig.show()

In [ ]:
# Las categorias más frecuentes en los tweets de los usarios de control
# Sumar las frecuencias de cada categoría
sums = df_count_A.sum()

# Crear un DataFrame con las sumas ordenadas de forma descendente
df_top_A = pd.DataFrame({'Categoría': sums.index, 'Frecuencia': sums.values}).sort_values(by='Frecuencia', ascending=False)
df_top_A.head(10)

,Categoría,Frecuencia
33,meccog,71772
66,verbos,66749
45,present,53233
22,funct,51327
48,relativ,50816
1,afect,46457
54,social,41508
17,emopos,27682
57,tiempo,23826
42,percept,23564
